<a href="https://colab.research.google.com/github/Dark-Sied/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent Recognition Dolores V01
Using a new vocabulary and training set.  
Intent Recognition is based on Intent_classification_final
Created by Christoph Windheuser, April 2020

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [ ]:
# DEFINE GLOBAL VARIABLES:
NUM_SENT = 0
NUM_INTENTS = 0
NUM_INTENTS_UNIQUE = 0
VOCABULARY_SIZE = 0
MAX_SENT_LENGTH = 0

In [64]:
# SHOW GLOBAL VARIABLES
print ("NUM_SENT:           ", NUM_SENT)
print ("NUM_INTENTS:        ", NUM_INTENTS)
print ("NUM_INTENTS_UNIQUE: ", NUM_INTENTS_UNIQUE)
print ("VOCABULARY_SIZE:    ", VOCABULARY_SIZE)
print ("MAX_SENT_LENGTH:    ", MAX_SENT_LENGTH)

NUM_SENT:            32
NUM_INTENTS:         32
NUM_INTENTS_UNIQUE:  8
VOCABULARY_SIZE:     39
MAX_SENT_LENGTH:     5


# Read the data

In [112]:
df = pd.read_csv("Dolores_Dataset_v01.csv", encoding = "latin1", names = ["Sentence", "Intent"])
print(df.head())
intents            = df["Intent"]
NUM_INTENTS        = len(list(df["Intent"]))
intents_unique     = list(set(df["Intent"]))
NUM_INTENTS_UNIQUE = len(intents_unique)
sentences          = list(df["Sentence"])
NUM_SENT           = len(sentences)



               Sentence      Intent
0          Good Morning   greetings
1                    Hi   greetings
2  Good morning Dolores   greetings
3           Hey Dolores   greetings
4                 Hello   greetings


In [113]:
print ("Unique Intents: ")
print (intents_unique)
print ("Num of unique Intents: ", len(intents_unique))


Unique Intents: 
[' getTime', ' saveAppoint', ' greetings', ' toSetDownSomeOfThisNaturalSplendor', ' haveViolentEnds', ' getDate', ' saveLink', ' goodBye']
Num of unique Intents:  8


In [13]:
df.head()

,Sentence,Intent
0,Good Morning,greetings
1,Hi,greetings
2,Good morning Dolores,greetings
3,Hey Dolores,greetings
4,Hello,greetings


In [14]:
df.shape

(32, 2)

In [15]:
print(sentences[:5])

['Good Morning', 'Hi', 'Good morning Dolores', 'Hey Dolores', 'Hello']


# Word Cleaning
re.sub is a routine from the "Regular Expression" Library.     
r'string' means that this is a "raw string", where backslashes are treated as charachters.    
re.sub(r'[^ a-z A-Z 0-9]', " ", s) means that all characters exept a-z, A-Z and 0-9 will be replaced by space.

In [20]:
clean_sent = []
for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    clean_sent.append([i.lower() for i in w])

print (clean_sent[:5])
print("Len of clean_sent: ", len(clean_sent))


[['good', 'morning'], ['hi'], ['good', 'morning', 'dolores'], ['hey', 'dolores'], ['hello']]
Len of clean_sent:  32


### Documentation for Tokenizer:
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [65]:
token = Tokenizer(filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
token.fit_on_texts(clean_sent)
VOCABULARY_SIZE = len(token.word_index) + 1
MAX_SENT_LENGTH = len(max(clean_sent, key = len))

#print("Vocab Size = %d. Maximum sent length = %d" % (vocab_size, max_sent_length))
print ("Max sent length: ", MAX_SENT_LENGTH)
encoded_sent = token.texts_to_sequences(clean_sent)
padded_sent = pad_sequences(encoded_sent, maxlen = max_sent_length, padding = "post")


Max sent length:  5


In [66]:
print (encoded_sent)
print ("Num of Sentences: ", len(encoded_sent))

[[10, 14], [24], [10, 14, 4], [25, 4], [15], [15, 4], [5], [5, 5], [26], [10, 5], [16, 11], [16, 11, 4], [5, 4], [6, 17, 3, 9], [6, 9, 18, 27], [9], [28, 19, 3, 9], [6, 18, 3, 7, 29], [6, 7, 20, 21, 22], [30, 7, 20, 21, 22], [6, 17, 3, 7], [31, 32, 33], [11, 34, 35], [1, 2, 12], [13, 1, 2, 12], [1, 3, 36, 12], [1, 3, 37, 7], [1, 2, 8, 38, 19], [1, 2, 8], [13, 1, 2, 8], [1, 2, 23, 8, 13], [1, 2, 23, 8]]
Num of Sentences:  32


In [67]:
padded_sent[:5]

array([[10, 14,  0,  0,  0],
       [24,  0,  0,  0,  0],
       [10, 14,  4,  0,  0],
       [25,  4,  0,  0,  0],
       [15,  0,  0,  0,  0]], dtype=int32)

In [68]:
print("Shape of padded sent = ",padded_sent.shape)

Shape of padded sent =  (32, 5)


# Tokenizing the intents

In [69]:
#tokenizer for the intents
token_intents = Tokenizer(filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
token_intents.fit_on_texts(intents_unique)


In [70]:
token_intents.word_index

{'gettime': 1,
 'saveappoint': 2,
 'greetings': 3,
 'tosetdownsomeofthisnaturalsplendor': 4,
 'haveviolentends': 5,
 'getdate': 6,
 'savelink': 7,
 'goodbye': 8}

In [71]:
encoded_output = token_intents.texts_to_sequences(intents)


In [72]:
print (encoded_output)


[[3], [3], [3], [3], [3], [3], [8], [8], [8], [8], [8], [8], [8], [1], [1], [1], [1], [6], [6], [6], [6], [5], [4], [2], [2], [2], [2], [7], [7], [7], [7], [7]]


In [73]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [74]:
# print (encoded_output)
type (encoded_output)

numpy.ndarray

In [75]:
encoded_output.shape

(32, 1)

In [76]:
one_hot = OneHotEncoder(sparse = False)
output_one_hot = one_hot.fit_transform(encoded_output)

In [77]:
output_one_hot.shape

(32, 8)

# Define Training- and Testset

In [78]:
from sklearn.model_selection import train_test_split

In [79]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_sent, output_one_hot, shuffle = True, test_size = 0.1)


In [80]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (28, 5) and train_Y = (28, 8)
Shape of val_X = (4, 5) and val_Y = (4, 8)


# Defining the Model

In [81]:
model = Sequential()
model.add(Embedding(VOCABULARY_SIZE, 128, input_length = MAX_SENT_LENGTH, trainable = False))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(NUM_INTENTS_UNIQUE, activation = "softmax"))

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 128)            4992      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
Total params: 276,648
Trainable params: 271,656
Non-trainable par

# Training the Model

In [82]:
import time

filename = 'dir_01.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

start = time.time()

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

print("Elapsed time in seconds: ", time.time() - start)


Instructions for updating:
Use tf.cast instead.
Train on 28 samples, validate on 4 samples
Epoch 1/100
28/28 [==============================] - 1s 46ms/step - loss: 2.0759 - acc: 0.1429 - val_loss: 2.0838 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 2.08376, saving model to dir_01.h5
Epoch 2/100
28/28 [==============================] - 0s 453us/step - loss: 2.0711 - acc: 0.2143 - val_loss: 2.0827 - val_acc: 0.0000e+00

Epoch 00002: val_loss improved from 2.08376 to 2.08274, saving model to dir_01.h5
Epoch 3/100
28/28 [==============================] - 0s 520us/step - loss: 2.0583 - acc: 0.5357 - val_loss: 2.0803 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from 2.08274 to 2.08034, saving model to dir_01.h5
Epoch 4/100
28/28 [==============================] - 0s 611us/step - loss: 2.0506 - acc: 0.5000 - val_loss: 2.0754 - val_acc: 0.0000e+00

Epoch 00004: val_loss improved from 2.08034 to 2.07540, saving model to dir_01.h5
Epoch 5/100
28/28 [===============


Epoch 00038: val_loss did not improve from 1.86550
Epoch 39/100
28/28 [==============================] - 0s 511us/step - loss: 1.3346 - acc: 0.6429 - val_loss: 1.9904 - val_acc: 0.0000e+00

Epoch 00039: val_loss did not improve from 1.86550
Epoch 40/100
28/28 [==============================] - 0s 642us/step - loss: 1.2938 - acc: 0.5357 - val_loss: 1.9849 - val_acc: 0.0000e+00

Epoch 00040: val_loss did not improve from 1.86550
Epoch 41/100
28/28 [==============================] - 0s 1ms/step - loss: 1.5331 - acc: 0.4286 - val_loss: 1.9660 - val_acc: 0.0000e+00

Epoch 00041: val_loss did not improve from 1.86550
Epoch 42/100
28/28 [==============================] - 0s 857us/step - loss: 1.2211 - acc: 0.4643 - val_loss: 1.9468 - val_acc: 0.0000e+00

Epoch 00042: val_loss did not improve from 1.86550
Epoch 43/100
28/28 [==============================] - 0s 903us/step - loss: 1.2922 - acc: 0.5714 - val_loss: 1.9309 - val_acc: 0.0000e+00

Epoch 00043: val_loss did not improve from 1.86550



Epoch 00082: val_loss did not improve from 1.84133
Epoch 83/100
28/28 [==============================] - 0s 947us/step - loss: 0.3153 - acc: 0.9286 - val_loss: 2.3712 - val_acc: 0.5000

Epoch 00083: val_loss did not improve from 1.84133
Epoch 84/100
28/28 [==============================] - 0s 1ms/step - loss: 0.5028 - acc: 0.8571 - val_loss: 2.3779 - val_acc: 0.5000

Epoch 00084: val_loss did not improve from 1.84133
Epoch 85/100
28/28 [==============================] - 0s 646us/step - loss: 0.3639 - acc: 0.8929 - val_loss: 2.4047 - val_acc: 0.5000

Epoch 00085: val_loss did not improve from 1.84133
Epoch 86/100
28/28 [==============================] - 0s 653us/step - loss: 0.3935 - acc: 0.9286 - val_loss: 2.4544 - val_acc: 0.5000

Epoch 00086: val_loss did not improve from 1.84133
Epoch 87/100
28/28 [==============================] - 0s 806us/step - loss: 0.2543 - acc: 0.9643 - val_loss: 2.5250 - val_acc: 0.5000

Epoch 00087: val_loss did not improve from 1.84133
Epoch 88/100
28/28 [

In [83]:
 model = load_model("dir_01.h5")

In [91]:
def predictions(text):
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    test_word = word_tokenize(clean)
    test_word = [w.lower() for w in test_word]
    test_ls = token.texts_to_sequences(test_word)
    #print(test_word)
    #Check for unknown words
    if [] in test_ls:
        test_ls = list(filter(None, test_ls))
    
    test_ls = np.array(test_ls).reshape(1, len(test_ls))

    #print("test_ls: ", test_ls)

    x = pad_sequences(test_ls, maxlen = MAX_SENT_LENGTH, padding = "post")
    
    #print ("x: ", x)
    
    pred = model.predict_proba(x)
  
    return pred


In [92]:
def get_final_output(pred, classes):
    #print (type (pred))
    #print (pred)
    
    predictions = pred[0]
 
    classes = np.array(classes)
    ids = np.argsort(-predictions)
    classes = classes[ids]
    predictions = -np.sort(-predictions)

    for i in range(pred.shape[1]):
        print("%s has confidence = %s" % (classes[i], (predictions[i])))


In [127]:
text = "You headin out"
pred = predictions(text)
get_final_output(pred, intents_unique)

 goodBye has confidence = 0.3582559
 greetings has confidence = 0.28471038
 getTime has confidence = 0.16156594
 toSetDownSomeOfThisNaturalSplendor has confidence = 0.084267005
 saveLink has confidence = 0.03471032
 saveAppoint has confidence = 0.03294208
 getDate has confidence = 0.028806573
 haveViolentEnds has confidence = 0.014741814


In [ ]:
' getTime', ' saveAppoint', ' greetings', 
' toSetDownSomeOfThisNaturalSplendor', 
'haveViolentEnds', ' getDate', ' saveLink', ' goodBye'

